In [5]:
import platform
print('Python Version = ',platform.python_version())

Python Version =  3.5.2


In [21]:
import sys,os
import dlib
import face_alignment
import cv2
import numpy as np
import tqdm

In [7]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import time

In [8]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, enable_cuda=True, flip_input=False)

In [9]:
def vid2img(filename):
    cap=cv2.VideoCapture(filename)
    success,frame=cap.read()
    allImg=[]
    while(success):
        allImg.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        success,frame=cap.read()
    return allImg

In [24]:
def face_align(src_img,dst_img,face_alignment_obj=fa):
#     img0 = cv2.resize(src_img,(256,256))
    img0 = src_img
    img1 = dst_img
    h,w,d = dst_img.shape
    fa_img0 = face_alignment_obj.get_landmarks(img0)[0]
    fa_img1 = face_alignment_obj.get_landmarks(img1)[0]
    
    src_img = img0
    src_points = fa_img0[:36, :2]
    dst_img = img1
    dst_points = fa_img1[:36, :2]
#     print(len(dst_points))
    
    H, status = cv2.findHomography(src_points, dst_points)
    warped_img_0_to_1 = cv2.warpPerspective(src_img, H, (w, h))
#     plt.figure(figsize=(10, 10))
#     plt.subplot(221)
#     plt.imshow(img0)
#     plt.subplot(222)
#     plt.imshow(warped_img_0_to_1)
#     plt.subplot(223)
#     plt.imshow(img1)
    lm_w_0_to_1 = np.round(face_alignment_obj.get_landmarks(warped_img_0_to_1)[0]).astype('int')
    
    lip_center_src=np.mean([lm_w_0_to_1[62][:2],lm_w_0_to_1[66][:2]],0)
    lc_src=np.round(lip_center_src).astype('int')
    lip_center_dst=np.mean([fa_img1[62][:2],fa_img1[66][:2]],0)
    lc_dst=np.round(lip_center_dst).astype('int')
    buffer_factor=1.1
    dist=np.round(np.sqrt(np.sum(np.square(lm_w_0_to_1[33,:2]-lc_src)))*buffer_factor).astype('int')
    img_trans=img1
    img_trans[lc_dst[1]-dist:lc_dst[1]+dist,lc_dst[0]-dist:lc_dst[0]+dist]= warped_img_0_to_1[lc_src[1]-dist:lc_src[1]+dist,lc_src[0]-dist:lc_src[0]+dist]
    return(img_trans)

In [25]:
def allImg2vid(allImg, output_vid='output_body.mp4', frameRate=30):
    vidCodec = cv2.VideoWriter_fourcc(*'XVID')
    height, width, channel = allImg[0].shape
    vidFile = cv2.VideoWriter(output_vid, vidCodec, frameRate, (width, height))
    for i in tqdm.tqdm(range(0,len(allImg))):
        frame = cv2.cvtColor(allImg[i],cv2.COLOR_RGB2BGR)
        vidFile.write(frame)
    vidFile.release()
    print(" Successfully converted images in allImg to "+output_vid)
    return None

In [27]:
def get_homography_video(target_video_fps, making_video_filename, target_video_filename, input_video_filenme, fa=fa):
#     keep target_video_fps=30 (default) for Andrew Ng
#     making_video_filename has four face sections in same video
#     target_video_filename is video with body
#     input_video_filename is used for audio transfer
    allImg_im0=vid2img(making_video_filename)
    allImg_im1=vid2img(target_video_filename)
    min_len=min([len(allImg_im0),len(allImg_im1)])
    
    h, w, d=allImg_im1[0].shape
    allImg_trans=np.zeros((min_len, h, w, d)).astype('uint8')
    for i in tqdm.tqdm(range(0,min_len)):
        allImg_trans[i]=face_align(allImg_im0[i][256:,:256],allImg_im1[i],face_alignment_obj=fa)
    
    allImg2vid(allImg_trans, frameRate=target_video_fps)
    shell_script='ffmpeg -i output_body.mp4 -i '+input_video_filenme+' -c copy -map 0:v:0 -map 1:a:0 -shortest output_with_body.mp4'
    os.system(shell_command)

In [ ]:
inputVid='input_videos/CV_01_C4W1L01_000003_to_000045/CV_01_C4W1L01_000003_to_000045_dynamic_lipsync_eng.mp4_hindi_abhishek_making.mp4'
outputVid='input_videos/CV_01_C4W1L01_000003_to_000045/CV_01_C4W1L01_000003_to_000045.mp4'


get_homography_video(25, inputVid, outputVid, inputVid)


  0%|          | 4/1107 [00:05<25:26,  1.38s/it]
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/abhishek/.local/lib/python3.5/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

 31%|███       | 338/1107 [07:47<17:44,  1.38s/it]

In [23]:
inputVid='input_videos/CV_01_C4W1L01_000003_to_000045/CV_01_C4W1L01_000003_to_000045_dynamic_lipsync_eng.mp4_hindi_abhishek_making.mp4'
shell_script='ffmpeg -i output_body.mp4 -i '+inputVid+' -c copy -map 0:v:0 -map 1:a:0 -shortest output_with_body.mp4'
os.system(shell_script)

0